# Variational approach to continuum thermodynamics

Gradient-driven minimization of the total energy potential which includes Maxwell viscoelasticity

Consider a Helmholtz free energy as $\Psi(\varepsilon, \mathcal{E})$ with $\varepsilon$ representing the external strain variable and $\mathcal{E}$ a vector of internal variables.

The rate of the Helmholtz potential is given as
$$
\dot{\Psi} = \frac{\partial \Psi}{\partial \varepsilon} \dot{\varepsilon}
           + \frac{\partial \Psi}{\partial \mathcal{E}} \dot{\mathcal{E}}
           = \frac{\partial \Psi}{\partial \varepsilon} \dot{\varepsilon}
           - \mathcal{S} : \dot{\mathcal{E}}
           = \frac{\partial \Psi}{\partial \varepsilon} \dot{\varepsilon}
           - \mathcal{D}(\mathcal{E}, \dot{\mathcal{E}}),
$$
with $\mathcal{S}$ and $\mathcal{D}$ representing the thermodynamic forces and the dissipation rate, respectively.

## Example - damage-viscoelastic model

Let us now consider a viscoelastic model of Maxwell type with the Helmholtz free energy defined as
$$
\Psi = \frac{1}{2} (1 - \omega) E (\varepsilon - \varepsilon_\mathrm{v})^2,
$$
where $\varepsilon_\mathrm{v}$ and $\omega$  represent the internal variables $\mathcal{E} = [\varepsilon_\mathrm{v}, \omega]$.
The dissipaion rate $\mathcal{D}$ is given as
$$
\mathcal{D} = 
\frac{\partial \Psi}{\partial \mathcal{E}} : \dot{\mathcal{E}}
=
\frac{\partial \Psi}{\partial \varepsilon_\mathrm{v}} : \dot{\varepsilon_\mathrm{v}}
+
\frac{\partial \Psi}{\partial \omega} : \dot{\omega}
=
(1 - \omega) E (\varepsilon - \varepsilon_\mathrm{v}) \dot{\varepsilon_\mathrm{v}} 
-
\frac{1}{2} E (\varepsilon - \varepsilon_\mathrm{v})^2 \dot{\omega}
= \sigma \dot{\varepsilon_\mathrm{v}}
+ Y \dot{\omega}.
$$

In the considered case of Maxwell viscoelasticity, we know that the strain rate at the constant stress level is inversely proportianal to the viscosity modulus $\eta$, i.e. 
$$
\dot{\varepsilon}_\mathrm{v} = \frac{\sigma}{\eta} \implies \sigma = \eta \dot{\varepsilon}_\mathrm{v}.
$$

This means that
$$
\mathcal{D} = \eta \dot{\varepsilon}_\mathrm{v}^2.
$$  

To construct the dissipation potential defining the evolution equations, let us recall that 
$$
\mathcal{S} = \frac{\partial \Phi}{\dot{\mathcal{E}}}.
$$
Thus, to device the MPDM with a compatible dissipation potential we set 
$$
\Phi = \frac{1}{2} \eta \dot{\varepsilon_\mathrm{v}}^2 + (1 - \omega)^c \frac{S}{r+1}\left(\frac{Y}{S}\right)^{r+1}.
$$

In [ ]:
%matplotlib widget
import sympy as sp
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from bmcs_utils.api import Cymbol
sp.init_printing()

In [ ]:
# Define symbols
from re import S
Pi, Psi, Phi, sig = sp.symbols('Pi Psi Phi sig')
t, E, eta = sp.symbols('t E eta')
sig, eps, eps_v, omega= sp.symbols('sigma epsilon epsilon_v, omega')
eps_v = 0
c, r, S = sp.symbols('c r S')
dot_eps = Cymbol(r'\dot{\varepsilon}', codename='dot_eps')
# Define the total strain as the sum of elastic and viscous parts
eps_e_ = eps - eps_v
dot_eps_v = Cymbol(r'\dot{\varepsilon}_\mathrm{v}', codename='dot_eps_v')
# Define Helmholtz free energy Psi
Psi_ = sp.Rational(1,2) * (1-omega) * E * eps_e_**2

In [ ]:
#sig_v_ = sp.simplify(sp.diff(Psi_, eps_v))
Y_ = -sp.diff(Psi_, omega)
Y_

In [ ]:
Y = Cymbol(r'Y', codename='Y')
dot_omega = Cymbol(r'\dot{\omega}', codename='dot_omega')
dot_omega_ = (1 - omega) * (Y / S)
dot_omega_eq = sp.Eq(dot_omega, dot_omega_)
Y_ = sp.solve(dot_omega_eq, Y)[0]
Phi_Y_ = sp.integrate(Y_, (dot_omega, 0, dot_omega))
dot_omega_eq, Y_, Phi_Y_

In [ ]:
# Define dissipation potential Phi
r = 1
#Phi_ = sp.Rational(1,2) * eta * dot_eps_v**2 + (1-omega)**c * S/(r+1) * (Y_/S)**(r+1)
# Phi_ = sp.Rational(1,2) * eta * dot_eps_v**2 + Phi_Y_
Phi_ = Phi_Y_
# Define the stress-strain relationship for the elastic part
sig_ = E * eps_e_
{Psi: Psi_, Phi: Phi_, sig: sig_}

## Time discretization and integration

In [ ]:
dt = Cymbol('\Delta t', codename='dt')
eps_n = Cymbol(r'\varepsilon^{(n)}', codename='eps_n')
dot_eps_n = Cymbol(r'\dot{\varepsilon}^{(n)}', codename='dot_eps_n')
eps_v_n = Cymbol(r'\varepsilon_\mathrm{v}^{(n)}', codename='eps_v_n')
dot_eps_v_n = Cymbol(r'\dot{\varepsilon}_\mathrm{v}^{(n)}', codename='dot_eps_v_n')
omega_n = Cymbol(r'\omega^{(n)}', codename='omega_n')
dot_omega_n = Cymbol(r'\dot{\omega}^{(n)}', codename='dot_omega_n')


The minimization of the dissipation potential is done for an increment $\Delta \Pi$ defined in the time interval $\Delta t$. This requires an integration over the increment $t \in (t_n, t_{n+1})$, where $t_{n+1} = t_{n} + \Delta t$.

$$
\Delta \Pi = \int_{t_n}^{t_{n+1}} \dot{\Psi} \, \mathrm{d}t
+ \int_{t_n}^{t_{n+1}} \dot{\Phi} \, \mathrm{d}t
$$

$$
\Delta \Pi = \Psi(t_{n+1}) - \Psi(t_n) 
+ \int_{t_n}^{t_{n+1}} \dot{\Phi} \, \mathrm{d}t
$$

$$
\Delta \Pi = \Psi(\varepsilon_{n+1}, \mathcal{E}_{n+1}) 
           - \Psi(\varepsilon_{n}, \mathcal{E}_{n}) 
+ \Delta t \Phi(\mathcal{E}_n, \dot{\mathcal{E}}_n)
$$


In [ ]:
# Substitute time discretization and integration into the expression for increment of potential energy
delta_Pi_ = (Psi_.subs({eps: eps_n + dot_eps_n * dt, eps_v: eps_v_n + dot_eps_v_n * dt, omega: omega_n + dot_omega_n * dt}) 
           - Psi_.subs({eps: eps_n, eps_v: eps_v_n, omega: omega_n}) 
           + dt * Phi_.subs({
               eps: eps_n + dot_eps_n * dt, eps_v: eps_v_n + dot_eps_v_n * dt, 
               omega: omega_n, dot_eps_v: dot_eps_v_n,
               dot_omega: dot_omega_n}))
delta_Pi_

In [ ]:
ddelta_Pi_dEps_ = sp.Matrix([
    sp.diff(delta_Pi_, Eps_i) for Eps_i in (dot_eps_v_n, dot_omega_n)])
ddelta_Pi_dEps_ = sp.Matrix([
    sp.diff(delta_Pi_, Eps_i) for Eps_i in (dot_omega_n,)])
# Lambdify the expressions
get_delta_Pi = sp.lambdify((eps_n, dot_eps_n, omega_n, dot_omega_n, dt, E, eta, c, S), 
                           delta_Pi_, 'numpy', cse=True)
get_ddelta_Pi_dEps = sp.lambdify((eps_n, dot_eps_n, omega_n, dot_omega_n, dt, E, eta, c, S), 
                                       ddelta_Pi_dEps_, 'numpy', cse=True)
delta_Pi_, ddelta_Pi_dEps_

In [ ]:
fig, ax = plt.subplots(1,1)
dot_omega_range = np.linspace(-50, 50, 200)
ax.plot(dot_omega_range, get_delta_Pi(0.01, 0, 0.5, dot_omega_range, 0.01, 30000, 3000, 1, 0.1), color='green', label='Pi_t_dt')
# ax.plot(eps_v_range, get_ddelta_Pi_ddot_eps_v(0.1, 0, 0, eps_v_range, 0.01, 30000, 0.1), color='orange', label='dPi_t_dt_ddot_eps_v')
# ax.plot(eps_v_range, get_delta_D(0.1, 0, 0, eps_v_range, 0.01, 30000, 3000), color='blue', label='D_')
ax.legend() 

In [ ]:
# Time integrator for viscoelasticity
def time_integrator(eps_t, d_t, E, eta, c, S):
    d_eps_t = np.diff(eps_t, axis=0)
    d_t_t = d_t * np.ones_like(d_eps_t)
    dot_eps_t = d_eps_t / d_t_t

    n_steps = len(eps_t)
    eps_v_t = np.zeros(n_steps)
    omega_t = np.zeros(n_steps)
    sig_t = np.zeros(n_steps)
    dot_eps_v_next = 0
    dot_omega_next = 0

    for i, d_t in enumerate(d_t_t):
        
        def objective(x):
            dot_omega, = x
#            Pi_val = get_delta_Pi(eps_t[i], dot_eps_t[i], eps_v_t[i], dot_eps_v, omega_t[i], dot_omega, d_t, E, eta, c, S)
            Pi_val = get_delta_Pi(eps_t[i], dot_eps_t[i], omega_t[i], dot_omega, d_t, E, eta, c, S)
            return Pi_val
        
        def gradient(x):
            dot_omega, = x
#            return np.array([get_ddelta_Pi_dEps(eps_t[i], dot_eps_t[i], eps_v_t[i], dot_eps_v, omega_t[i], dot_omega, d_t, E, eta, c, S)])
            return np.array([get_ddelta_Pi_dEps(eps_t[i], dot_eps_t[i], omega_t[i], dot_omega, d_t, E, eta, c, S)])
        
        res = minimize(objective, [dot_omega_next],
                       jac=gradient,
                       bounds=[ (0, None)], 
                       method='L-BFGS-B')
        
        dot_omega_next, = res.x
        eps_v_t[i+1] = eps_v_t[i] + dot_eps_v_next * dt
        omega_t[i+1] = omega_t[i] + dot_omega_next * dt
        sig_t[i+1] = (1 - omega_t[i+1]) * E * (eps_t[i+1] - eps_v_t[i+1])
    
    return eps_t, eps_v_t, omega_t, sig_t

In [ ]:
# Example usage
E_val = 210e3  # Young's modulus in MPa
eta_val = 10e+2  # Viscosity modulus in Pa / s
S_val = 0.000000001  # Damage rate
c_val = 2  # Damage rate exponent
dt = 0.01  # Time step in seconds
final_time = 10  # Final time in seconds

# Generate cyclic sinusoidal loading history
time = np.arange(0, final_time, dt)
eps_t_2_cycles = 0.01 * np.sin(2 * np.pi * 1 * time / final_time)  # 2 cycles
eps_t_4_cycles = 0.01 * np.sin(2 * np.pi * 4 * time / final_time)  # 4 cycles

# Compute responses for both loading histories
eps_t_2, eps_v_t_2, omega_t_2, sig_t_2 = time_integrator(eps_t_2_cycles, dt, E_val, eta_val, c_val, S_val)
eps_t_4, eps_v_t_4, omega_t_4, sig_t_4 = time_integrator(eps_t_4_cycles, dt, E_val, eta_val, c_val, S_val)
eps_e_t_2 = eps_t_2 - eps_v_t_2
eps_e_t_4 = eps_t_4 - eps_v_t_4

In [ ]:
# Plot results
plt.figure(figsize=(8, 12))

plt.subplot(4, 1, 1)
plt.plot(time, eps_t_2_cycles, label='Total Strain (2 cycles)')
plt.plot(time, eps_e_t_2, label='Elastic Strain (2 cycles)')
plt.plot(time, eps_v_t_2, label='Viscous Strain (2 cycles)')
ax = plt.twinx()
ax.plot(time, omega_t_2, color='red', ls='dashed', label='Damage')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.legend()
plt.title('Strain Components for 2 Cycles')

plt.subplot(4, 1, 2)
plt.plot(time, eps_t_4_cycles, label='Total Strain (4 cycles)')
plt.plot(time, eps_e_t_4, label='Elastic Strain (4 cycles)')
plt.plot(time, eps_v_t_4, label='Viscous Strain (4 cycles)')
plt.xlabel('Time [s]')
plt.ylabel('Strain')
plt.legend()
plt.title('Strain Components for 4 Cycles')

plt.subplot(4, 1, 3)
plt.plot(time, sig_t_2, label='Stress (2 cycles)')
plt.plot(time, sig_t_4, label='Stress (4 cycles)')
plt.xlabel('Time [s]')
plt.ylabel('Stress [Pa]')
plt.legend()
plt.title('Stress Response for Different Loading Rates')

plt.subplot(4, 1, 4)
plt.plot(eps_t_2_cycles, sig_t_2, label='Stress-strain (2 cycles)')
plt.plot(eps_t_4_cycles, sig_t_4, label='Stress-Strain (4 cycles)')
plt.xlabel('Strain [-]')
plt.ylabel('Stress [Pa]')
plt.legend()
plt.title('Stress Response for Different Loading Rates')

plt.tight_layout()
plt.show()

In [ ]:
# Let's visualize the stress versus strain curves from the previous example using matplotlib
plt.figure(figsize=(8, 6)) # Create a new figure
plt.plot(eps_t_2, sig_t_2, label='Stress-strain (2 cycles)') # Plot the stress-strain curve for 2 cycles
plt.plot(eps_t_4, sig_t_4, label='Stress-strain (4 cycles)') # Plot the stress-strain curve for 4 cycles 
plt.xlabel('Strain [-]') # Set the x-axis label
plt.ylabel('Stress [Pa]') # Set the y-axis label
plt.legend() # Show the legend
plt.title('Stress-strain response for different loading rates') # Set the title
plt.show() # Show the plot
